<a href="https://colab.research.google.com/github/georgez9/programming_tips/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! pip install d2l

In [15]:
import torch
from torch import nn
from d2l import torch as d2l

$(n_h-k_h+1) \times (n_w-k_w+1)$


In [24]:
def corr2d(X, K):
  """ 计算二维互相关运算 """
  # X: 输入张量，K: 卷积核
  h, w = K.shape
  # 在二维互相关运算中，输出尺寸由输入和卷积核的尺寸决定
  # 没有填充（padding）且步幅（stride）为 1 的情况下，输出尺寸计算公式为上述形式
  Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
  return Y

In [25]:
class Conv2D(nn.Module):
  """ 2D卷积层定义 """
  # nn.Module是所有神经网络的基类
  def __init__(self, kernel_size):
    super().__init__()  # 父类初始化
    self.weight = nn.Parameter(torch.rand(kernel_size)) # 权重
    self.bias = nn.Parameter(torch.zeros(1))  # 偏置项

  def forward(self, x): # 二维互相关运算
    return corr2d(x, self.weight) + self.bias

## 实例：边缘检测

In [33]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [34]:
# 当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零
K = torch.tensor([[1.0, -1.0]])

In [35]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

若要检查水平边缘，需要转置

In [36]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 如何自动查找一个卷积核

In [37]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
# in_channels=1, out_channels=1
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率，决定参数更新步长

for i in range(10):
    Y_hat = conv2d(X)
    # 计算预测输出与目标输出之间的均方误差（MSE），l为损失张量
    l = (Y_hat - Y) ** 2
    # 每次反向传播之前，需要将之前的梯度清零
    conv2d.zero_grad()
    # 将损失张量的所有元素求和，得到一个标量损失值，
    # 对标量损失进行反向传播，以计算损失对卷积层参数的梯度
    # 这一步是自动计算参数的梯度，并存储在conv2d.weight.grad中
    l.sum().backward()
    # 迭代卷积核
    # 梯度下降公式：new_weight=old_weight−learning_rate×gradient
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    # 每隔 2 个迭代（即第 2、4、6、8、10 次迭代）输出一次日志
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 5.947
epoch 4, loss 1.437
epoch 6, loss 0.421
epoch 8, loss 0.144
epoch 10, loss 0.054


结果：

In [38]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0128, -0.9661]])